In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.7 MB/s eta 0:00:00


- 데이터 구하기 단계

In [2]:
from datasets import load_dataset #Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment") # 데이터 다운로드

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Korean_sentiment-c4fbcaaf8f2ffcfa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print("Data type: ", type(data)) #데이터 타입 확인
print("Data structure: ", data) #데이터 구조 확인
print("Data keys: ", data.keys()) #데이터 키 확인

print(data['train'][0]) #데이터 타입 확인

Data type:  <class 'datasets.dataset_dict.DatasetDict'>
Data structure:  DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 36000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2667
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1333
    })
})
Data keys:  dict_keys(['train', 'test', 'validation'])
{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


- 라벨이 1인 이유는 평점이 긍정이기 때문에 준 것으로 보인다.
- 라벨이 0이면 평점이 부정적!

In [4]:
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

print(train_data)
print(dev_data)
print(test_data)

# #형태소 단위로 Tokenization 된 텍스트를 {train/dev/test}_data에 저장
# train_data = train_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
# dev_data = dev_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
# test_data = test_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
# # example은 본인 자체를 나타낸다. lambda라는 거는 함수인데 런타임 도중 함수 실행하는 거라고 한다. 그래서 의미는 label은 원래 label 그대로 넣고, text 데이터는 형태소 분석한 텍스트로 대체한다.
# #조인을 하면 단어 사이에 ' ' 띄어쓰기를 넣겠다라는 것
# #역시 명작 어렸을 때 ==> 역시 명작 어렸 을 때 이렇게 형태소 단위로 띄어버린다.

Dataset({
    features: ['label', 'text'],
    num_rows: 36000
})
Dataset({
    features: ['label', 'text'],
    num_rows: 1333
})
Dataset({
    features: ['label', 'text'],
    num_rows: 2667
})


- 데이터 준비 단계

In [5]:
#### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 하는 법 구현)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # CountVectorizer : Bag of words 벡터화 하기 위한 클래스

vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,1)) #데이터를 벡터화 해주는 모델
vectorizer.fit(train_data['text']) # fit 진행시켜! 학습 시키는 것, 텍스트 문서 모음을 토큰 수의 행렬로 변환 => 문자 -> 수치화

print(vectorizer.vocabulary_) # 한 번 어휘 집합을 출력 

print("---------------------")

print("어휘 집합 길이 : ", len(vectorizer.vocabulary_)) # 어휘 집합의 길이도 출력


train_vectors = vectorizer.transform(train_data['text'])
dev_vectors = vectorizer.transform(dev_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



111111dddfedafdssa 태소 분석을 하니 토큰 수가 100,494에서 26,879로 바뀌더라
형태소 분석 전에는 어절로 분석했다.

 봤을 때도, 먹었을 때도 ==> 봤을 때 도, 먹었 을 때 도 == 토큰의 개수가 확 줄었다.


이전에는 먹었을때도 먹었을때도 봤을 때도를 아예 다른 토큰으로 만든다. 봤을지라도, 봤을건데 전부다 다른 토큰 수로 추가해야 한다.


근데 봤을 때 도 하면은 토큰에 추가해야 하는 단어가 줄어든다. 단어들이 더 쪼개 지면서 재사용이 될 수 있어짐. 전체 토큰이 10만개에서 2만개로 줄어든 것이다!! 이해 되었다!


In [6]:
print(train_data[0]['text']) # 결과 확인
print("---------------------")
print(train_vectors) # 결과 확인

역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요
---------------------
  (0, 82955)	0.24803531244759022
  (0, 76600)	0.3586563080553068
  (0, 76567)	0.26583172003536687
  (0, 70669)	0.3653684561951893
  (0, 61807)	0.22189503499966104
  (0, 59095)	0.31451462517036477
  (0, 41839)	0.37816376511131566
  (0, 41511)	0.24536839594296717
  (0, 32878)	0.43386436732096745
  (0, 32639)	0.2570503765929463
  (1, 70018)	0.2457411522112294
  (1, 63206)	0.2633538098403386
  (1, 62328)	0.557295664912992
  (1, 58651)	0.5363695019797161
  (1, 30035)	0.5215221530938965
  (2, 66997)	0.4566376215403588
  (2, 50412)	0.47445308874076364
  (2, 48177)	0.47445308874076364
  (2, 43188)	0.3995492730612046
  (2, 18965)	0.42618187920056516
  (3, 86461)	0.39868423280802956
  (3, 84021)	0.1607167661522813
  (3, 79557)	0.4142386802251096
  (3, 77893)	0.30128414885264015
  (3, 69342)	0.16748916268165
  :	:
  (35994, 68519)	0.5200338255432987
  (35994, 58532)	0.6285027308687494
  (35994, 50302)	0.5784022281953332
  (35995, 9468

In [7]:
print(train_data[35999]['text'])

큰 의미없이 연결'만' 되는 살인들.


In [8]:
print(train_data['label'])

[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 

In [9]:
sample_num = 0 #확인하고 싶은 샘플 번호
sample_origin = train_data[sample_num] #확인하고 싶은 샘플의 원래 문장
print("Original Input: ", sample_origin) 
print("------------------------------------------------------")
sample_transform = train_vectors[sample_num]
print(sample_transform) # 확인하고 싶은 샘플의 transform된 결과
#CountVectorizer는 count만 나오는데 tfdif방식은 가중치를 볼 수 있더라 tf-idf의 로그 계산 한 거 나오는 것
print("------------------------------------------------------")
sample_inverse_transfrom = vectorizer.inverse_transform(sample_transform) # 확인하고 싶은 샘플의 transform된 결과를 다시 단어의 조합으로 바꾼 결과
print(sample_inverse_transfrom)

Original Input:  {'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}
------------------------------------------------------
  (0, 82955)	0.24803531244759022
  (0, 76600)	0.3586563080553068
  (0, 76567)	0.26583172003536687
  (0, 70669)	0.3653684561951893
  (0, 61807)	0.22189503499966104
  (0, 59095)	0.31451462517036477
  (0, 41839)	0.37816376511131566
  (0, 41511)	0.24536839594296717
  (0, 32878)	0.43386436732096745
  (0, 32639)	0.2570503765929463
------------------------------------------------------
[array(['지금', '재밌었고', '재밌어요', '이상으로', '역시',
       '어렸을때', '봤을때도', '봐도', '몇억배', '명작'],
      dtype='<U338')]


In [10]:
print(train_data[-1])
print(train_vectors[-1])
print(vectorizer.inverse_transform(train_vectors[-1]))
# 5주차에서 unicode 넣고 단어 사라짐 문제 해결!!


{'label': 0, 'text': "큰 의미없이 연결'만' 되는 살인들."}
  (0, 90189)	0.3118234738558217
  (0, 69198)	0.4424379551583432
  (0, 61905)	0.4424379551583432
  (0, 45735)	0.4727867319536863
  (0, 25056)	0.30328995160893885
  (0, 91)	0.4424379551583432
[array(['큰', '의미없이', '연결', '살인들', '되는', "'만"],
      dtype='<U338')]


- 선택 훈련 단계

In [11]:
from sklearn.svm import LinearSVC # LinearSVC: linear SVM을 사용하기 위한 클래스
svm = LinearSVC() # 모델 정의
svm.fit(train_vectors, train_data['label'])

print(svm.coef_) # weights (w)
print("--------------------")
print(svm.intercept_)  # bias (b)

[[-0.31194619 -0.33987566  0.1750026  ... -0.34580546  0.11423304
  -0.00327867]]
--------------------
[-0.07857303]


fit을 할 때는 데이터와 label를 넣어줘야한다.

- 교차 검증을 사용한 평가

In [12]:
# from sklearn.model_selection import cross_val_score

# all_data = train_data['text']+dev_data['text']+test_data['text'] # all_data = train_data + dev_data + test_data 의 text 다 합쳤다.
# all_label = train_data['label']+dev_data['label']+test_data['label'] # all_label은 train_data, dev_data, test_date의 레이블을 모두 합한 것
# all_vectors = vectorizer.transform(all_data) # all_data(텍스트)를 숫자로 변환 (벡터화) 그걸 올 벡터
# scores = cross_val_score(svm, all_vectors, all_label, cv=5) # 5-cross validation - cv는 몇 번 할거냐 5개로 데이터 나누고 1개 테스트로 시킨다. 이게 커질수록 학습이 오래 걸린다.

# print("All scores: ", scores) # 전체 점수 출력
# print("Average: {: .2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
# print("Standard deviation: {:.6f}".format(scores.std())) # 표준편차 출력

In [13]:
# scores = cross_val_score(svm, all_vectors, all_label, cv=10) # 10-cross validation - cv는 몇 번 할거냐 10개로 데이터 나누고 1개 테스트로 시킨다. 이게 커질수록 학습이 오래 걸린다.

In [14]:
# print("All scores: ", scores) # 전체 점수 출력
# print("Average: {: .2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
# print("Standard deviation: {:.6f}".format(scores.std())) # 표준편차 출력

In [15]:
# from sklearn.model_selection import GridSearchCV
# param_grid = [{'max_iter': [500, 1000, 5000], 'C': [1, 10, 100]}] # 학습 max_iter와 C를 변동 하이퍼파라미터로 설정
# # 500이고 1일때, 1000이고 10일 때 5000이고 100일 때
# # Regularization parameter 그냥 알아만 둬라 C라는 값을 크게 하면 ~~ 
# grid_search = GridSearchCV(svm, param_grid, cv=3) #하이퍼파라미터 서치할 모델 설정, 각 모델별로 5-cross validation 실행
# grid_search.fit(train_vectors, train_data['label'])  # transformed train data로 학습

# print(grid_search.cv_results_['mean_test_score']) # 파라미터 서치 결과
# print(grid_search.best_params_) # 최고의 모델 파라미터

# #파라미터 서치 결과 출력

- 3번 검증했을 때 9개의 성능으로 나온다.
c가 1이고 max가 500일 때 최고의 하이퍼 파라미터 성능이더라
- grid search cv가 무슨 값이라고 하셨지? - 교차검증을 몇 번 돌릴지 5-cross이면 cv=5 이런 거 같다!
그래서 5번의 값이 나오고 그것의 평균도 구하고 한다


In [16]:
from sklearn.metrics import accuracy_score # Accuracy 측정 함수 import

# final_model = grid_search.best_estimator_ # grid search에 의해 정해진 제일 좋은 하이퍼파라미터
final_model = svm

pred_results = final_model.predict(test_vectors) # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results) # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100)) #정확도 출력

Accuracy: 80.88%
